# AI - Fall 00 - Hands-on - Genetic Algorithm

<div style="font-size: 16px">
<b>Paria Khoshtab 810198387</b>
<hr>
</div>

## Goal

The goal of this project is to get more familiar with `genetic algorithms`, `mating`, `mutation`, and natural `selection` techniques.
We also learn that sometimes by choosing simple natural selection criteria, these algorithms perform poorly and
We should consider a selection criterion that, in addition to individual performance, also cares about the `diversity`
of the population.

## Brief Description

In this problem, we are asked to solve a `sudoku puzzle` using a `genetic algorithm`. Sudoku is a puzzle based on a
small number of very simple rules: Every square has to contain a single number. Only the numbers from 1 through to
9 can be used. Each 3×3 box can only contain a number from 1 to 9 once. Each vertical column can only contain
a number from 1 to 9 once.

## Modeling the Problem

In genetic algorithms, first, we have to provide a definition for the `gene` and then use it to create a `chromosome`. 
Each chromosome is a suggested way to solve the problem.

In this problem, we assume a `gene` is a **cell** in the sudoku puzzle
and a `chromosome` is the **entire sudoku puzzle**, which actually contains 81 genes.<hr>

First we have to import necessary libraries.

In [1]:
import numpy as np
import random
import copy
import random
import time

Then we define some constants like the row and column of the Soduko puzzle and the size of the population in each generation of
the genetic algorithm. We consider the population size to be 800.

In [2]:
SIZE = 800
ROW = 9
COL = 9

In the code below, we read each line of the input file and save the given cells of the Soduku puzzle in a 2D array.

In [3]:
initial_grid = np.arange(ROW * COL).reshape(ROW, COL)
file = open("Test2.txt")
for i in range(ROW):
    initial_grid[i] = file.readline().split()

We create a class called `Chromosome` containing `genes` and `fitness` of a chromosome Which includes 3 methods for updating fitness, mutation
(we will cover it later), and printing genes.

In [4]:
class Chromosome:
    def __init__(self, genes):
        self.genes = genes
        self.fitness = 0
        self.update_fitness()
        
    def update_fitness(self):
        self.fitness = 0
        for row in self.genes:
            self.fitness += len(np.unique(row))
            
        for column in self.genes.transpose():
            self.fitness += len(np.unique(column))
            
        for i in range(0, ROW, ROW // 3):
            for j in range(0, COL, COL // 3):
                self.fitness += len(np.unique(self.genes[i: i + 3, j: j + 3]))        
        
    def print_genes(self):
        print(self.genes)
        
    def mutate(self):
        self.update_fitness()
        for i in range(ROW):
            for j in range(COL):
                if initial_grid[i][j] == 0:                    
                    r = random.uniform(0, 1)                                           
                    valid_numbers = []
                    square = self.genes[(i // 3) * 3: (i // 3) * 3 + 3, (j // 3) * 3: (j // 3) * 3 + 3]
                    column = self.genes.transpose()[j]
                    row = self.genes[i]
                    for num in range(1, 10):
                        if num not in square and num not in column and num not in row and num != self.genes[i][j]:
                            valid_numbers.append(num)
                    if len(valid_numbers) > 0 and r < (1 - 1 / (9 * len(valid_numbers))):
                        self.genes[i][j] = random.choices(valid_numbers)[0]


## Initialization

We make the `initial population` of chromosomes completely `randomly` and 
without any special orientation. The size of this population, as mentioned above, is considered 800.

**generate_random_chromosome:** This function generates a random chromosome by randomly
filling all cells(genes) in the Sudoku puzzle (chromosome) except the given cells with specific numbers.

**generate_initial_population:** This function generates an initial population of chromosomes by calling
generate_random_chromosome function.

In [6]:
def generate_random_chromosome():
    chromosome = copy.deepcopy(initial_grid)
    for i in range(ROW):
        for j in range(COL):
            if chromosome[i][j] == 0:
                chromosome[i][j] = random.randint(1, 9)      
    return Chromosome(chromosome)

def generate_initial_population():
    population = []
    for i in range(SIZE):
        population.append(generate_random_chromosome())
    return population

## Fitness Function

We consider the `fitness function` as the total number of unique numbers in each row, column, and 3*3 square in the puzzle which 
is an integer number from 1 to 243. A chromosome with fitness = 243 is the final solution to the problem.

In [5]:
def set_fitness(children):
    for child in children:
        child.update_fitness()
    return children

## Selection

We use `rank-based selection`:
The rank-based selection first ranks the population and then every chromosome receives fitness from this ranking.
The worst will have fitness 1, the second worst 2, etc. and the best will have fitness N (number of chromosomes in population).
After this, all the chromosomes have a chance to be selected.
Rank-based selection schemes can avoid premature convergence, resulting in a diverse population.

First, we pass 20% of the high-fitness chromosomes directly to the next generation and then implement rank-based selection
as follows:

1. Accumulated normalized fitness values are computed: the accumulated fitness value of an individual is 
the sum of its own fitness value plus the fitness values of all the previous individuals.

2. A random number r between 0 and 1 is chosen.

3. The selected individual is the first one whose accumulated normalized value is greater than or equal to r.

In [7]:
def selection(population):
    mating_pool = []
    for i in range(80*SIZE//100, SIZE):
        mating_pool.append(population[i])
    accumulated_normalized_value = []
    sum_rank = SIZE * (SIZE + 1) / 2
    accumulated_normalized_value = [(i*(i+1)/2)/sum_rank for i in range (1, SIZE + 1)]        
    while len(mating_pool) < SIZE:
        r = random.random()
        for j in range(len(accumulated_normalized_value)):
            if accumulated_normalized_value[j] >= r:
                mating_pool.append(population[j]);
                break                
    return mating_pool

## Crossover

We apply a `1-point crossover` on each row of the chromosome. 
In each row, a point on both parents' chromosomes is picked randomly, and designated a `crossover point`.
numbers to the right of that point are swapped between the two parent chromosomes. 
This results in two offspring, each carrying some genetic information from both parents.

We set the `crossover rate` at 0.9 to combine a large percentage of parents each time using the crossover function.

**crossover:** This function generates two new children using the 1-point method on each row of chromosomes.

**crossover_on_population:** This function calls the crossover function in order to generate new children with a rate of 0.9,
it means that if a random float number from 0 to 1 is less than 0.9, we do a crossover, otherwise, we transfer random parents
directly to the next step.

In [8]:
def crossover(parent1, parent2):
    child1 = copy.deepcopy(parent1.genes)
    child2 = copy.deepcopy(parent2.genes)        
    for i in range(ROW):
        point = random.randint(1, 7)
        for j in range(point, COL):
            child1[i][j] = parent2.genes[i][j]
            child2[i][j] = parent1.genes[i][j]        
    return Chromosome(child1), Chromosome(child2)
    
    
def crossover_on_population(population):
    children = []
    while len(children) < SIZE:
        r = random.uniform(0, 1)
        parent1 = random.choices(population)[0]
        parent2 = random.choices(population)[0]
        if r < 0.9:
            child1, child2 = crossover(parent1, parent2)
            children.append(child1)
            children.append(child2)
        else:
            children.append(parent1)
            children.append(parent2)
    return children

## Mutation

In this function, we iterate all the genes of chromosomes and assign each gene a `random valid number`
(Non-duplicate numbers in the corresponding row, column, and square)

We consider the `mutation rate` to be 1-1/(9*len(valid_numbers)).

*Note that mutation function is defined in chromosome class above.

In [9]:
def mutate_on_children(children):
    for child in children:
        child.mutate()
    return children

## Genetic Algorithm

Finally we implement genetic algorithm, using implemented functions above.

In [10]:
def genetic_algorithm():
    population = generate_initial_population()
    generation = 1
    while(True):
        print("Generation:", generation)
        population = set_fitness(population)
        population.sort(key = lambda chromosome: chromosome.fitness)
        print("Best Fitness:", population[SIZE - 1].fitness)
        if population[SIZE - 1].fitness == 243:
            population[SIZE - 1].print_genes()
            break
        population = selection(population)
        children = crossover_on_population(population)
        children = mutate_on_children(children)
        population = copy.deepcopy(children)
        generation += 1          

In [11]:
start = time.time()
genetic_algorithm()
end = time.time()
print('execution time', end-start)

Generation: 1
Best Fitness: 180
Generation: 2
Best Fitness: 213
Generation: 3
Best Fitness: 225
Generation: 4
Best Fitness: 224
Generation: 5
Best Fitness: 229
Generation: 6
Best Fitness: 235
Generation: 7
Best Fitness: 238
Generation: 8
Best Fitness: 238
Generation: 9
Best Fitness: 243
[[9 6 8 2 5 3 4 7 1]
 [4 7 5 1 9 6 3 8 2]
 [3 1 2 4 8 7 6 9 5]
 [2 5 1 9 4 8 7 6 3]
 [7 9 3 6 2 5 8 1 4]
 [8 4 6 3 7 1 2 5 9]
 [1 8 7 5 3 2 9 4 6]
 [6 2 9 8 1 4 5 3 7]
 [5 3 4 7 6 9 1 2 8]]
execution time 33.61712408065796


## Questions

<b>1) Explain the method of selecting the superior chromosomes to produce the next
population and the reason for choosing the method used.</b><br>
As fully explained above, we use rank-based selection because this method can lead to slower convergence 
and the best chromosomes do not differ so much from other ones.

<b>2) State the reason for choosing your fitness function.</b><br>
Since the solution of the Soduko puzzle contains 9 unique values in each row, column, and square, the total number of unique
numbers in each row, column, and square in the puzzle is a good choice because it can be easily computed efficiently and
it has a definite answer(243), with which we can find the final solution.<br><br>

<b>3) State the effect of crossover and mutation functions, the probability of each of them, and 
the reason for choosing the probability value.</b><br>
Crossover: crossover is used to generate new solutions from an existing population, in order to produce 
chromosomes similar to the optimal solution.<br>
we consider a constant crossover rate of 0.9, in order to crossover a large percentage of parents each time using the crossover function
and have a diverse population.<br>
Mutation: mutation is used to maintain genetic diversity from one generation of a population of genetic algorithm
chromosomes to the next, in order to avoid premature convergence.<br>
we consider dynamic mutation rate 1-1/9*len(valid_numbers) which is obtained by trial and error
and testing different mutation rates. As discussed in class, this rate is higher in the beginning when
we are far from the optimal solution, and the closer we get to the main solution, the lower the rate.<br>

<b>4) Chromosomes may not change after a few more steps. Explain the reason for this and the problems it creates.
What solution do you suggest to solve it?</b><br>
Reason: Population is filled by individuals that represent the suboptimal solution which is (too) close to the optimal
solution.<br>
Problems: Genetic algorithm converges to local optima and even after a lifetime, 
the algorithm does not reach the optimal solution.<br>
Solutions: Changing hyperparameters like population size, mutation rate, crossover rate, and selection rate.

## Conclusion

Genetic algorithms are commonly used to generate high-quality solutions to optimization and search problems by 
relying on operators such as mutation, crossover, and selection.
This algorithm is faster and easier than other algorithms because we can find out a solution without a deep analysis work 
in problems with a large number of solutions' successors, but
the random heuristics sometimes doesn’t find the optimum, So it is not a complete algorithm 
(not always the algorithm finds a suitable solution)and sometimes it can get stuck with a local maximum/minimum problem.
Nevertheless, crossover and mutation operations help to mitigate it, although this implies
more iterations and time.